# Czym wgl jest NLP?
![Czym jest NLP](https://7wdata.be/wp-content/uploads/2021/09/DCF_NLP-in-the-Data-Center_ML-DL-Diagram.png)

NLP - Natural Language Processing to dział AI zajmujący się przetwarzaniem języka naturalnego. Czyli takie rzeczy jak:
* Generowanie artykułów
* Podsumowywanie tekstu
* POS (part of speach tagging)
* Przetwarzanie zbiorów danych
* Automatyczne Question anwsering
itp... Ogólnie wszystko powiązanego z językami, którymi się posługują ludzie.
Na codzień mamy doczynienia z tekstem więc nic dziwnego, że jest to tak istotna i prężnie rozwijająca się działka AI.
W związku z tym że korzystamy z komputerów pojawiają się problemy jak reprezentować ten tekst, jak go przetwarzać.

# Jak reprezentować tekst - podstawowe sposoby

## OneHotEncoding
Najprostszym sposobem jest już znany wam pewnie OneHotEncoding. Robimy to w następujący sposób:
1) tworzymy słownik ze słowami zawierającymi się w naszym zbiorze $V=(słowo_1,słowo_2,\dots,słowo_k)$
2) reprezentujemy dany tekst na podstawie wektora 0 i 1, gdzie 0 dajemy jeżeli wystąpiło dane słowo
czyli 
$$T = (a_1,a_2,\dots,a_{|V|}) \quad \text{gdzie } a_i =\begin{cases} 0 \text{ gdy }słowo_i\not\in Tekst\\  1 \text{ gdy }słowo_i\in Tekst\end{cases}$$
Gdzie V to nasz słownik, a T to reprezentacja.


In [3]:
def create_vocab(text):
    words = text.split()
    vocab = list(set(words))
    return vocab

def encode_text(text, vocab):
    text_words = set(text.split())
    return [word in text_words for word in vocab]

In [4]:
text = "Ala ma psa"
text2 = "Maciek nie ma psa"

vocab = create_vocab(text)
encode_text(text2, vocab), vocab

([True, False, True], ['psa', 'Ala', 'ma'])

### Bag of Words
Wadą naszego porzedniego rozwiązania jest to, że tylko sprawdza czy słowo wystąpiło, dlatego teraz dodatkowo będziemy zliczali liczbę wystąpień słów.
1) tworzymy słownik ze słowami zawierającymi się w naszym zbiorze $V=(słowo_1,słowo_2,\dots,słowo_k)$
2) reprezentujemy dany tekst na podstawie wektora wystąpień słów ze słownika w tekście
czyli 
$$T = (a_1,a_2,\dots,a_{|V|}) \quad \text{gdzie } a_i-\text{liczba wystąpień }słowa_i\text{ w tekście}$$
Gdzie V to nasz słownik, a T to reprezentacja.

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import fetch_20newsgroups

In [6]:
twenty_train = fetch_20newsgroups(subset='train', shuffle=True, random_state=42)

In [7]:
twenty_train.data[0]

"From: lerxst@wam.umd.edu (where's my thing)\nSubject: WHAT car is this!?\nNntp-Posting-Host: rac3.wam.umd.edu\nOrganization: University of Maryland, College Park\nLines: 15\n\n I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.\n\nThanks,\n- IL\n   ---- brought to you by your neighborhood Lerxst ----\n\n\n\n\n"

In [8]:
vectorizer = CountVectorizer().fit(twenty_train.data)

In [9]:
vectorizer.vocabulary_

{'from': 56979,
 'lerxst': 75358,
 'wam': 123162,
 'umd': 118280,
 'edu': 50527,
 'where': 124031,
 'my': 85354,
 'thing': 114688,
 'subject': 111322,
 'what': 123984,
 'car': 37780,
 'is': 68532,
 'this': 114731,
 'nntp': 87620,
 'posting': 95162,
 'host': 64095,
 'rac3': 98949,
 'organization': 90379,
 'university': 118983,
 'of': 89362,
 'maryland': 79666,
 'college': 40998,
 'park': 92081,
 'lines': 76032,
 '15': 4605,
 'was': 123292,
 'wondering': 124931,
 'if': 65798,
 'anyone': 28615,
 'out': 90774,
 'there': 114579,
 'could': 42876,
 'enlighten': 51793,
 'me': 80638,
 'on': 89860,
 'saw': 104813,
 'the': 114455,
 'other': 90686,
 'day': 45295,
 'it': 68766,
 'door': 48618,
 'sports': 109581,
 'looked': 76718,
 'to': 115475,
 'be': 32311,
 'late': 74693,
 '60s': 16574,
 'early': 50111,
 '70s': 18299,
 'called': 37433,
 'bricklin': 34995,
 'doors': 48620,
 'were': 123796,
 'really': 99822,
 'small': 108252,
 'in': 66608,
 'addition': 26073,
 'front': 56989,
 'bumper': 35612,
 'se

In [10]:
transformed = vectorizer.transform(twenty_train.data[:1])
transformed

<1x130107 sparse matrix of type '<class 'numpy.int64'>'
	with 89 stored elements in Compressed Sparse Row format>

In [11]:
vectorizer = CountVectorizer().fit(['Ala ma psa'])

In [12]:
transformed = vectorizer.transform(['Maciek ma psa i ma wiele kotów, które nie lubią psów'])

In [13]:
transformed.toarray()
# Zauważcie pierwszy problem, psów i psy mają tak naprawdę to samo znaczenie ale przez to
# że są w innej odmianie nie są zliczane.

array([[0, 2, 1]])

### TF-IDF (Term Frequency Inverse Document Frequency)
Zauważmy, że wadą zliczania jest to, że mogą występować w naszym zbiorze teksty, w których często powtarza się
to samo słowo i wtedy to że np. słowo "paragraf" wystąpiło 2137 za dużo nie znaczy. Z pomocą nadchodzi TF-IDF, który waży liczności
w zależności od tego w ilu dokumentach dane słowo wystąpiło, czyli większe wagi chcemy przywiązywać słowom "rozróżniającym". Niech t oznacza token, d oznacza dokument oraz D-zbiór dokumentów wtedy
\begin{align*}
    &tf(t,d)=\frac{f_{t,d}}{\sum_{t'\in d} f_{t',d}}\\
    &idf(t,D)=\log\frac{N}{|\{d\in D;\,t\in d\}}\\
    &tfidf(t,d,D)=tf(t,d)\times idf(t,D)
\end{align*}

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [15]:
vectorizer = TfidfVectorizer().fit(twenty_train.data)
transformed_data = vectorizer.transform(twenty_train.data)

In [16]:
transformed_data

<11314x130107 sparse matrix of type '<class 'numpy.float64'>'
	with 1787565 stored elements in Compressed Sparse Row format>

#### Challenge!
Korzystając z przedstawionych wcześniej reprezentacji tekstu przeucz drzewo losowe przewidujące klasę newsa

In [19]:
twenty_test = fetch_20newsgroups(subset='test', shuffle=True, random_state=42)
train_text, train_class = twenty_train.data, twenty_train.target
test_text, test_class = twenty_test.data, twenty_test.target

In [ ]:
#Rozwiązanie


### n-gramy

Zauważmy, że czasem istotną informacją mogą być ciągi słów. Na tym właśnie polegają n-gramy. N-gram jest to ciąg n-słów.

Przykład 2-grama: \["Ala", "ma"\], \["ma", "Psa"\]

In [21]:
from nltk.util import ngrams

In [23]:
text = "Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud "

In [25]:
# funckja ngrams wymaga podania na wejście sekwencji
list(ngrams(text.split(), 3))

[('Lorem', 'ipsum', 'dolor'),
 ('ipsum', 'dolor', 'sit'),
 ('dolor', 'sit', 'amet,'),
 ('sit', 'amet,', 'consectetur'),
 ('amet,', 'consectetur', 'adipiscing'),
 ('consectetur', 'adipiscing', 'elit,'),
 ('adipiscing', 'elit,', 'sed'),
 ('elit,', 'sed', 'do'),
 ('sed', 'do', 'eiusmod'),
 ('do', 'eiusmod', 'tempor'),
 ('eiusmod', 'tempor', 'incididunt'),
 ('tempor', 'incididunt', 'ut'),
 ('incididunt', 'ut', 'labore'),
 ('ut', 'labore', 'et'),
 ('labore', 'et', 'dolore'),
 ('et', 'dolore', 'magna'),
 ('dolore', 'magna', 'aliqua.'),
 ('magna', 'aliqua.', 'Ut'),
 ('aliqua.', 'Ut', 'enim'),
 ('Ut', 'enim', 'ad'),
 ('enim', 'ad', 'minim'),
 ('ad', 'minim', 'veniam,'),
 ('minim', 'veniam,', 'quis'),
 ('veniam,', 'quis', 'nostrud')]

Wszystkie wcześniej wymienione sposoby reprezentacji tekstu domyślnie operują na słowach ale mogą operować też na n-gramach

In [26]:
vectorizer = CountVectorizer(ngram_range=(1,2)).fit(['Ala ma psa'])
transformed = vectorizer.transform(['Maciek ma psa i ma wiele kotów, które nie lubią psów'])

In [29]:
vectorizer.get_feature_names()

['ala', 'ala ma', 'ma', 'ma psa', 'psa']

In [31]:
transformed.toarray()

array([[0, 0, 2, 1, 1]])

Analogicznie dla TFIDF

Wracamy do poprzedniego zadania, spróbujcie poprawić wynik wykorzystując n-gramy, tylko nie przesadzajcie z n bo wam nie starczy RAMu

In [32]:
twenty_test = fetch_20newsgroups(subset='test', shuffle=True, random_state=42)
train_text, train_class = twenty_train.data, twenty_train.target
test_text, test_class = twenty_test.data, twenty_test.target

In [ ]:
# Rozwiązanie


## Zmniejszanie słownika
### Stop words
W każdym języku znajdują się słowa, które same nie niosą żadnej informacji, czasami warto pozbyć się takich słów, aby zmniejszyć wymiar słownika itp.

Przykłady dla angielskiego: "The" "who"

In [33]:
from nltk.corpus import stopwords

In [36]:
stops = set(stopwords.words("english"))

In [37]:
stops

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

### Stemming i Lematyzacja

Jak wcześniej zwróciłem uwagę czasami nie interesuje nas odmiana słowa tylko czy samo słowo wystąpiło. Np. chcemy w tekście szukać czy wystąpiła jakakolwiek odmiana słowa "pies" wtedy dokonujemy stemmingu(usunięcia ostatnich znaków ze słowa aby sprowadzić je do podstawowej formy) lub lematyzaji wykorzystującej kontekst

In [52]:
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import nltk

In [47]:
nltk.download("wordnet")
nltk.download('omw-1.4')


[nltk_data] Downloading package wordnet to /home/mchraba/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/mchraba/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


True

In [53]:
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

In [58]:
words = [
    "caresses",
    "flies",
    "dies",
    "mules",
    "denied",
    "died",
    "agreed",
    "owned",
    "humbled",
    "sized",
    "meeting",
    "stating",
    "siezing",
    "itemization",
    "sensational",
    "traditional",
    "reference",
    "colonizer",
    "plotted",
]
for word in words:
    lemmatized, stemmed = lemmatizer.lemmatize(word), stemmer.stem(word)
    print(f"{word}\nLemma:{lemmatized} Stemmed:{stemmed}")


caresses
Lemma:caress Stemmed:caress
flies
Lemma:fly Stemmed:fli
dies
Lemma:dy Stemmed:die
mules
Lemma:mule Stemmed:mule
denied
Lemma:denied Stemmed:deni
died
Lemma:died Stemmed:die
agreed
Lemma:agreed Stemmed:agre
owned
Lemma:owned Stemmed:own
humbled
Lemma:humbled Stemmed:humbl
sized
Lemma:sized Stemmed:size
meeting
Lemma:meeting Stemmed:meet
stating
Lemma:stating Stemmed:state
siezing
Lemma:siezing Stemmed:siez
itemization
Lemma:itemization Stemmed:item
sensational
Lemma:sensational Stemmed:sensat
traditional
Lemma:traditional Stemmed:tradit
reference
Lemma:reference Stemmed:refer
colonizer
Lemma:colonizer Stemmed:colon
plotted
Lemma:plotted Stemmed:plot


## Regex